In [1]:
import twitter
from tqdm import tqdm
from copy import copy
from datetime import datetime
from time import strptime
import pandas as pd
import nltk
import numpy as np
from enum import Enum

In [2]:
api = twitter.Api(consumer_key='USctDCgnlL3bNRB6UuXn7d236',
                  consumer_secret='J2VViasvzh3joWrl8sHicoKLnHcjtWk9mM7O9wsieWyqy4W1PL',
                  access_token_key='782531360-ldm9KTYfSwyKXIq4Axh9Op6fDP0IarPCuIl6xzkh',
                  access_token_secret='vKDEmonUAxvL1M1YGvMXDL6OfNwvKgzTD9Se6qKsLRnjT')
api.VerifyCredentials()
long_lat_dict = {'helsinki':[60.2979104,25.0186232],
                 'moscow':[55.877528, 37.529321],
                 'kair':[30.192449, 31.171607]} 

In [46]:
def make_post(text):
    api.PostUpdate(text + '    randseed:'+ str(np.random.randint(1000)))

In [47]:
make_post('FAKE NEWS AGENCY big terrorist attack happened in SVO, Moscow right now!!')

make_post('FAKE NEWS AGENCY big explosion in the airport happened in SVO, Moscow right now')
make_post('FAKE NEWS AGENCY union strike happened in SVO, Moscow right now')
make_post('FAKE NEWS AGENCY labour strike in airport happened in SVO, Moscow right now')


In [24]:
a = dict()
res = a.update({'type':EVENT_TYPE.TERRORIST_ATTACK})

In [ ]:
de

In [44]:
res = api.PostUpdate('FAKE NEWS AGENCY {}!! big terrorist attack happened in right now!!!'.format(np.random.randint(100)),
                                           latitude=long_lat_dict['kair'][0],
                                           longitude=long_lat_dict['kair'][1],
                                           display_coordinates=True).AsDict()
res.update({'type':EVENT_TYPE.TERRORIST_ATTACK})
print(res)
Mention().gen_from_twitter(res,cities_set,icao_set,iata_set)

{'coordinates': {'coordinates': [31.171607, 30.192449], 'type': 'Point'}, 'user_mentions': [], 'id_str': '934708181892288517', 'created_at': 'Sun Nov 26 08:59:31 +0000 2017', 'hashtags': [], 'geo': {'coordinates': [30.192449, 31.171607], 'type': 'Point'}, 'lang': 'en', 'source': '<a href="http://exapmle.com" rel="nofollow">Test_bonlime_2</a>', 'text': 'FAKE NEWS AGENCY 27!! big terrorist attack happened in right now!!!', 'urls': [], 'place': {'country': 'مصر', 'attributes': {}, 'contained_within': [], 'name': 'القليوبية', 'place_type': 'admin', 'bounding_box': {'coordinates': [[[31.0544738, 30.1044313], [31.6086408, 30.1044313], [31.6086408, 30.6076595], [31.0544738, 30.6076595]]], 'type': 'Polygon'}, 'url': 'https://api.twitter.com/1.1/geo/id/01b64e0e77a035fc.json', 'id': '01b64e0e77a035fc', 'country_code': 'EG', 'full_name': 'القليوبية, مصر'}, 'id': 934708181892288517, 'user': {'profile_sidebar_fill_color': 'DDEEF6', 'listed_count': 3, 'created_at': 'Sun Aug 26 14:20:44 +0000 2012', 

datetime_happened None
datetime_reported 1511704771.0
longitude         31.3315573
lattitude         30.6076595
type              EVENT_TYPE.TERRORIST_ATTACK
airport_id        None
city_name         москва
country_name      مصر
raw_description   FAKE NEWS AGENCY 27!! big terrorist attack happened in right now!!!
tweet it          934708181892288517

In [ ]:
r = api.GetSearch(geocode=[12.2979104,15.2979104,'10mi'])
print(r)

In [43]:
import twitter
from tqdm import tqdm
from copy import copy
from datetime import datetime
from time import strptime
import pandas as pd
import nltk
import numpy as np
from enum import Enum

EVENT_TYPE = Enum('EVENT_TYPE', 'TERRORIST_ATTACK LABOUR_STRIKE')

class Mention():
    datetime_happened = None
    datetime_reported = None
    longitude = None
    lattitude = None
    type = None
    # Please use standard system: SVO, SCO, TSE
    airport_id = None
    city_name = None
    country_name = None
    raw_description = None
    
    @classmethod
    def gen_explicitly(cls, type, airport_id, datetime_reported, datetime_happened=None, lattitude=None, longitude=None, city_name=None, country_name=None, raw_description=None):
        m = Mention()
        m.datetime_happened = datetime_happened
        m.datetime_reported = datetime_reported
        m.longitude = longitude
        m.lattitude = lattitude
        m.type = type
        m.airport_id = airport_id
        m.city_name = None if city_name is None else city_name.lower()
        m.country_name = None if country_name is None else country_name.lower()
        m.raw_description = None
        return m
        
    @classmethod
    def gen_from_twitter(cls,status_dict,cities_set,icao_set,iata_set):
        m = Mention()       
        
        m.datetime_reported = pd.Timestamp.strptime(status_dict['created_at'],
                                               '%a %b %d %H:%M:%S +0000 %Y').timestamp()
        if 'utc_offset' in status_dict['user']:
            m.datetime_reported += status_dict['user']['utc_offset']
        
        if 'location' in status_dict['user']:
            m.city_name = status_dict['user']['location'].split(',')[0].lower()
        if 'place' in status_dict:
            if status_dict['place']['place_type'] == 'city':
                if m.city_name:
                    pass
                    #print('overwriting city {} with {}'.format(m.city_name,status_dict['place']['name']))
                m.city_name = status_dict['place']['name'].lower()
            if 'country' in status_dict['place']:
                m.country_name = status_dict['place']['country'].lower()
                # m.country_name = status_dict['place']['full_name'].split(',')[1].strip() в презе не нужно
            if status_dict['place']['place_type'] == 'country':
                pass # не нужно для презы
            if 'bounding_box' in status_dict['place']:
                m.longitude = (status_dict['place']['bounding_box']['coordinates'][0][2][0]+ \
                                  status_dict['place']['bounding_box']['coordinates'][0][3][0])/2.0
                m.lattitude = (status_dict['place']['bounding_box']['coordinates'][0][2][1]+ \
                                  status_dict['place']['bounding_box']['coordinates'][0][3][1])/2.0 # ??
        m.raw_description = status_dict['text']
        tokenized_text_set = set(nltk.word_tokenize(m.raw_description.lower()))
        cities_in_text = list(tokenized_text_set & cities_set)
        if cities_in_text:
            m.city_name = cities_in_text[0].lower()
            m.country_name = None
        airports_in_text = list( tokenized_text_set & (icao_set | iata_set) ) 
        if airports_in_text:
            m.airport_id = airports_in_text[0]
        m.type = status_dict['type']
        m.id = status_dict['id']
        return m
   
    def print_self(self):
        """" Obsolete function. Better use __repr__ instead """
        print("datetime_happened {}".format(self.datetime_happened))
        print("datetime_reported {}".format(self.datetime_reported))
        print("longitude         {}".format(self.longitude))
        print("lattitude         {}".format(self.lattitude))
        print("type              {}".format(self.type))
        print("airport_id        {}".format(self.airport_id))
        print("city_name         {}".format(self.city_name))
        print("country_name      {}".format(self.country_name))
        print("raw_description   {}".format(self.raw_description))
        
    def __repr__(self):        
        res = "\n".join(["datetime_happened {}".format(self.datetime_happened),
        "datetime_reported {}".format(self.datetime_reported),
        "longitude         {}".format(self.longitude),
        "lattitude         {}".format(self.lattitude),
        "type              {}".format(self.type),
        "airport_id        {}".format(self.airport_id),
        "city_name         {}".format(self.city_name),
        "country_name      {}".format(self.country_name),
        "raw_description   {}".format(self.raw_description),
        "tweet it          {}".format(self.id)])
        return res

class Event(Mention):
    mentions = []
    flights_affected = []
    
    def __init__(self, m:Mention):
        self.datetime_happened = m.datetime_happened
        self.datetime_reported = m.datetime_reported
        self.longitude = m.longitude
        self.lattitude = m.lattitude
        self.type = m.type
        self.airport_id = m.airport_id
        self.city_name = None if m.city_name is None else m.city_name.lower()
        self.country_name = None if m.country_name is None else m.country_name.lower()
        self.raw_description = None
        #super().__init__(m.type, m.airport_id, m.datetime_reported, m.datetime_happened, m.lattitude, m.longitude, m.city_name, m.country_name, None)
        self.mentions.append(m)
        
    def __eq__(self, other):
        return (self.airport_id == other.airport_id 
                and self.datetime_happened == other.datetime_happened 
                and self.type == self.type)
    
    def get_confidence():
        pass

In [2]:
class Mention():
    def __init__(self,status_dict,cities_set,icao_set,iata_set):
        self.datetime_happened = None
        self.datetime_reported = None
        self.longitude = None
        self.lattitude = None
        self.type = None
        self.airport_id = None
        self.city_name = None
        self.country_name = None
        self.raw_description = None
        
        
        self.datetime_reported = pd.Timestamp.strptime(status_dict['created_at'],
                                               '%a %b %d %H:%M:%S +0000 %Y').timestamp()
        if 'utc_offset' in status_dict['user']:
            self.datetime_reported += status_dict['user']['utc_offset']
        
        if 'location' in status_dict['user']:
            self.city_name = status_dict['user']['location'].split(',')[0]
        if 'place' in status_dict:
            if status_dict['place']['place_type'] == 'city':
                if self.city_name:
                    pass
                    #print('overwriting city {} with {}'.format(self.city_name,status_dict['place']['name']))
                self.city_name = status_dict['place']['name']
                # self.country_name = status_dict['place']['full_name'].split(',')[1].strip() в презе не нужно
            if status_dict['place']['place_type'] == 'country':
                pass # не нужно для презы
            if 'bounding_box' in status_dict['place']:
                self.longitude = (status_dict['place']['bounding_box']['coordinates'][0][2][0]+ \
                                  status_dict['place']['bounding_box']['coordinates'][0][3][0])/2.0
                self.lattitude = (status_dict['place']['bounding_box']['coordinates'][0][2][1]+ \
                                  status_dict['place']['bounding_box']['coordinates'][0][3][1])/2.0 # ??
        self.raw_description = status_dict['text']
        tokenized_text_set = set(nltk.word_tokenize(self.raw_description.lower()))
        cities_in_text = list(tokenized_text_set & cities_set)
        if cities_in_text:
            self.city_name = cities_in_text[0]
        airports_in_text = list( tokenized_text_set & (icao_set | iata_set) ) 
        if airports_in_text:
            self.airport_id = airports_in_text[0]
        self.type = status_dict['type']
        self.id = status_dict['id']
   
    def print_self(self):
        """" Obsolete function. Better use __repr__ instead """
        print("datetime_happened {}".format(self.datetime_happened))
        print("datetime_reported {}".format(self.datetime_reported))
        print("longitude         {}".format(self.longitude))
        print("lattitude         {}".format(self.lattitude))
        print("type              {}".format(self.type))
        print("airport_id        {}".format(self.airport_id))
        print("city_name         {}".format(self.city_name))
        print("country_name      {}".format(self.country_name))
        print("raw_description   {}".format(self.raw_description))
        
    def __repr__(self):        
        res = "\n".join(["datetime_happened {}".format(self.datetime_happened),
        "datetime_reported {}".format(self.datetime_reported),
        "longitude         {}".format(self.longitude),
        "lattitude         {}".format(self.lattitude),
        "type              {}".format(self.type),
        "airport_id        {}".format(self.airport_id),
        "city_name         {}".format(self.city_name),
        "country_name      {}".format(self.country_name),
        "raw_description   {}".format(self.raw_description),
        "tweet it          {}".format(self.id)])
        return res

class Twitter_parser():
    def __init__(self):
        self.api = twitter.Api(consumer_key='USctDCgnlL3bNRB6UuXn7d236',
                          consumer_secret='J2VViasvzh3joWrl8sHicoKLnHcjtWk9mM7O9wsieWyqy4W1PL',
                          access_token_key='782531360-ldm9KTYfSwyKXIq4Axh9Op6fDP0IarPCuIl6xzkh',
                          access_token_secret='vKDEmonUAxvL1M1YGvMXDL6OfNwvKgzTD9Se6qKsLRnjT')
        self.api.VerifyCredentials()
        self.airports_data = pd.read_csv('airports.csv')
        self.airports_data.city.fillna(value='')
        self.cities_set = set([str(i).lower() for i in self.airports_data['city'].values])
        self.icao_set = set(self.airports_data['icao'])
        self.iata_set = set(self.airports_data['iata'])
        #airports_data.head()
        self.key_words = ['terrorist attack', 'explosion in the airport','labour strike','union strike']
        self.key_words_last_ids = dict(zip(self.key_words,[None]*len(self.key_words)))
        print(self.key_words_last_ids)
        EVENT_TYPE = Enum('EVENT_TYPE', 'TERRORIST_ATTACK LABOUR_STRIKE')
        self.event_dict ={'terrorist attack':EVENT_TYPE.TERRORIST_ATTACK,
                         'explosion in the airport':EVENT_TYPE.TERRORIST_ATTACK,
                         'labour strike':EVENT_TYPE.LABOUR_STRIKE,
                         'union strike':EVENT_TYPE.LABOUR_STRIKE
                         }
        self.first_launch = True
    def parse_twitter(self):    
        all_res = []
        num_iterations = 5 # на практике это слишком много и не нужно
        #event_dict ={'terrorist attack':EVENT_TYPE.TERRORIST_ATTACK,'labour_strike':EVENT_TYPE.LABOUR_STRIKE}
        for key in self.key_words:
            print("collecting data for '{}' keyword".format(key))
            if self.first_launch:
                all_data = self.api.GetSearch(term=key,
                                              count=100,
                                              result_type='recent')
                self.first_launch = False
            else:
                all_data = self.api.GetSearch(term=key,
                                              count=100,
                                              result_type='recent',
                                              since_id=self.key_words_last_ids[key])
            if len(all_data) == 0:
                continue
            last_id = all_data[-1].AsDict()['id']
            for i in range(num_iterations): # заменить на 50
                all_data.extend(self.api.GetSearch(term=key,
                                                   count=100,
                                                   result_type='recent',
                                                   max_id=last_id-1,
                                                   since_id=self.key_words_last_ids[key]))
                last_id = all_data[-1].AsDict()['id']
            self.key_words_last_ids[key] = all_data[0].AsDict()['id']
            #all_res.append([])
            for tweet in all_data:
                if ('retweeted_status' not in tweet.AsDict().keys()) and ('quoted_status' not in tweet.AsDict().keys()):
                    dict_tweet = tweet.AsDict()
                    dict_tweet['type'] = self.event_dict[key]
                    all_res.append(Mention(dict_tweet,self.cities_set,self.icao_set,self.iata_set))
            #print('{}/{} tweets are not retweets'.format(len(all_res[-1]),num_iterations*100))
        return all_res

def delete_last_statuses(num_to_delete):
    for stat_id in [status.AsDict()['id'] for status in api.GetUserTimeline(count=num_to_delete)]:
        api.DestroyStatus(stat_id)
def make_post(text,coords):
    api.PostUpdate(text + '    randseed:'+ str(np.random.randint(1000)))



In [30]:
a = Twitter_parser()
all_men = a.parse_twitter()

{'labour strike': None, 'union strike': None, 'explosion in the airport': None, 'terrorist attack': None}
collecting data for 'terrorist attack' keyword
48
collecting data for 'explosion in the airport' keyword
51
collecting data for 'labour strike' keyword
232
collecting data for 'union strike' keyword
459
{'labour strike': 934566127241453568, 'union strike': 934566125760843776, 'explosion in the airport': 934566124271882240, 'terrorist attack': 934566977187786752}


In [37]:
a.parse_twitter()

collecting data for 'terrorist attack' keyword
1
collecting data for 'explosion in the airport' keyword
2
collecting data for 'labour strike' keyword
3
collecting data for 'union strike' keyword


TwitterError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [7]:
airports_data = pd.read_csv('airports.csv')
airports_data.city.fillna(value='')
cities_set = set([str(i).lower() for i in airports_data['city'].values])
icao_set = set(airports_data['icao'])
iata_set = set(airports_data['iata'])
#airports_data.head()
EVENT_TYPE = Enum('EVENT_TYPE', 'TERRORIST_ATTACK LABOUR_STRIKE')
key_words = ['terrorist attack', 'explosion in the airport','labour strike','union strike']
event_dict ={'terrorist attack':EVENT_TYPE.TERRORIST_ATTACK,
             'explosion in the airport':EVENT_TYPE.TERRORIST_ATTACK,
             'labour strike':EVENT_TYPE.LABOUR_STRIKE,
             'union strike':EVENT_TYPE.LABOUR_STRIKE}

In [34]:
class Mention():
    def __init__(self,status_dict):
        self.datetime_happened = None
        self.datetime_reported = None
        self.longitude = None
        self.lattitude = None
        self.type = None
        self.airport_id = None
        self.city_name = None
        self.country_name = None
        self.raw_description = None
        
        
        self.datetime_reported = pd.Timestamp.strptime(status_dict['created_at'],
                                               '%a %b %d %H:%M:%S +0000 %Y').timestamp()
        if 'utc_offset' in status_dict['user']:
            self.datetime_reported += status_dict['user']['utc_offset']
        
        if 'location' in status_dict['user']:
            self.city_name = status_dict['user']['location'].split(',')[0]
        if 'place' in status_dict:
            if status_dict['place']['place_type'] == 'city':
                if self.city_name:
                    pass
                    #print('overwriting city {} with {}'.format(self.city_name,status_dict['place']['name']))
                self.city_name = status_dict['place']['name']
                # self.country_name = status_dict['place']['full_name'].split(',')[1].strip() в презе не нужно
            if status_dict['place']['place_type'] == 'country':
                pass # не нужно для презы
            if 'bounding_box' in status_dict['place']:
                self.longitude = (status_dict['place']['bounding_box']['coordinates'][0][2][0]+ \
                                  status_dict['place']['bounding_box']['coordinates'][0][3][0])/2.0
                self.lattitude = (status_dict['place']['bounding_box']['coordinates'][0][2][1]+ \
                                  status_dict['place']['bounding_box']['coordinates'][0][3][1])/2.0 # ??
        self.raw_description = status_dict['text']
        tokenized_text_set = set(nltk.word_tokenize(self.raw_description.lower()))
        cities_in_text = list(tokenized_text_set & cities_set)
        if cities_in_text:
            self.city_name = cities_in_text[0]
        airports_in_text = list( tokenized_text_set & (icao_set | iata_set) ) 
        if airports_in_text:
            self.airport_id = airports_in_text[0]
        self.type = status_dict['type']
    def print_self(self):
        print("datetime_happened {}".format(self.datetime_happened))
        print("datetime_reported {}".format(self.datetime_reported))
        print("longitude         {}".format(self.longitude))
        print("lattitude         {}".format(self.lattitude))
        print("type              {}".format(self.type))
        print("airport_id        {}".format(self.airport_id))
        print("city_name         {}".format(self.city_name))
        print("country_name      {}".format(self.country_name))
        print("raw_description   {}".format(self.raw_description))
    def __repr__(self):        
        res = "\n".join(["datetime_happened {}".format(self.datetime_happened),
        "datetime_reported {}".format(self.datetime_reported),
        "longitude         {}".format(self.longitude),
        "lattitude         {}".format(self.lattitude),
        "type              {}".format(self.type),
        "airport_id        {}".format(self.airport_id),
        "city_name         {}".format(self.city_name),
        "country_name      {}".format(self.country_name),
        "raw_description   {}".format(self.raw_description)])
        return res

In [35]:
def parse_twitter(key_words):    
    all_res = []
    num_iterations = 5 # на практике это слишком много и не нужно
    #event_dict ={'terrorist attack':EVENT_TYPE.TERRORIST_ATTACK,'labour_strike':EVENT_TYPE.LABOUR_STRIKE}
    for key in key_words:
        print("collecting data for '{}' keyword".format(key))
        all_data = api.GetSearch(term=key,count=100,result_type='recent')
        last_id = all_data[-1].AsDict()['id']
        for i in tqdm(range(num_iterations)): # заменить на 50
            all_data.extend(api.GetSearch(term=key,count=100,result_type='recent',max_id=last_id-1))
            last_id = all_data[-1].AsDict()['id']
        #all_res.append([])
        for tweet in all_data:
            if ('retweeted_status' not in tweet.AsDict().keys()) and ('quoted_status' not in tweet.AsDict().keys()):
                dict_tweet = tweet.AsDict()
                dict_tweet['type'] = event_dict[key]
                all_res.append(Mention(dict_tweet))
        #print('{}/{} tweets are not retweets'.format(len(all_res[-1]),num_iterations*100))
    
    return all_res
def delete_last_statuses(num_to_delete):
    for stat_id in [status.AsDict()['id'] for status in api.GetUserTimeline(count=num_to_delete)]:
        api.DestroyStatus(stat_id)
def make_post(text):
    api.PostUpdate(text + '    randseed:'+ str(np.random.randint(1000)))

In [628]:
all_mention = parse_twitter(key_words)

collecting data for 'terrorist attack' keyword


  0%|          | 0/5 [00:00<?, ?it/s]

collecting data for 'explosion in the airport' keyword


100%|██████████| 5/5 [00:00<00:00,  7.88it/s]


collecting data for 'labour strike' keyword


100%|██████████| 5/5 [00:02<00:00,  2.36it/s]


collecting data for 'union strike' keyword


100%|██████████| 5/5 [00:02<00:00,  2.29it/s]


In [629]:
print(all_mention[119])

datetime_happened None
datetime_reported 1511444308.0
longitude         None
lattitude         None
type              EVENT_TYPE.LABOUR_STRIKE
airport_id        None
city_name         Scotland: Nation of Lions
country_name      None
raw_description   @bigbuachaille Fills me with disgust to see Labour pretending to champion firefighters. Their behaviour and insults… https://t.co/gjM5IMSwDP


In [615]:
for m in all_mention:
    print(m.print_self())
    print("")

datetime_happened None
datetime_reported 1511651584.0
longitude         None
lattitude         None
type              EVENT_TYPE.TERRORIST_ATTACK
airport_id        None
city_name         Cambridge
country_name      None
raw_description   @Reece6_88 @AntBrolia He didnt Celebrate for Cause of the terrorist Attack in Egypt https://t.co/dFqux0cM8G
None

datetime_happened None
datetime_reported 1511647928.0
longitude         None
lattitude         None
type              EVENT_TYPE.TERRORIST_ATTACK
airport_id        None
city_name         #WHU
country_name      None
raw_description   The terrorist attack in the Sinai had nothing to do with ‘poverty’ @bbcnews . It has to do purely with deranged minds of Islamist psychos
None

datetime_happened None
datetime_reported 1511647872.0
longitude         None
lattitude         None
type              EVENT_TYPE.TERRORIST_ATTACK
airport_id        None
city_name         Chandrakona
country_name      None
raw_description   @IndianExpress When our beautif

[Status(ID=934699231948038144, ScreenName=bonlime, Created=Sun Nov 26 08:23:57 +0000 2017, Text='FAKE NEWS AGENCY big terrorist attack happened in SVO, Moscow right now!!!'), Status(ID=934697672321191936, ScreenName=bonlime, Created=Sun Nov 26 08:17:45 +0000 2017, Text='FAKE NEWS AGENCY big terrorist attack happened in SVO, Moscow right now!!')]


In [41]:
r[0].AsDict()

{'coordinates': {'coordinates': [15.2979104, 12.2979104], 'type': 'Point'},
 'created_at': 'Sun Nov 26 08:23:57 +0000 2017',
 'geo': {'coordinates': [12.2979104, 15.2979104], 'type': 'Point'},
 'hashtags': [],
 'id': 934699231948038144,
 'id_str': '934699231948038144',
 'lang': 'en',
 'place': {'attributes': {},
  'bounding_box': {'coordinates': [[[13.4714404, 7.44392],
     [24.0000005, 7.44392],
     [24.0000005, 23.4462827],
     [13.4714404, 23.4462827]]],
   'type': 'Polygon'},
  'contained_within': [],
  'country': 'Chad',
  'country_code': 'TD',
  'full_name': 'Chad',
  'id': '9d86cf820e7f9341',
  'name': 'Chad',
  'place_type': 'country',
  'url': 'https://api.twitter.com/1.1/geo/id/9d86cf820e7f9341.json'},
 'source': '<a href="http://exapmle.com" rel="nofollow">Test_bonlime_2</a>',
 'text': 'FAKE NEWS AGENCY big terrorist attack happened in SVO, Moscow right now!!!',
 'urls': [],
 'user': {'created_at': 'Sun Aug 26 14:20:44 +0000 2012',
  'default_profile': True,
  'descriptio

In [39]:
r = r[0].AsDict()
r['type'] = EVENT_TYPE.TERRORIST_ATTACK
print(Mention(r))

datetime_happened None
datetime_reported 1511702637.0
longitude         18.73572045
lattitude         23.4462827
type              EVENT_TYPE.TERRORIST_ATTACK
airport_id        None
city_name         moscow
country_name      None
raw_description   FAKE NEWS AGENCY big terrorist attack happened in SVO, Moscow right now!!!


In [ ]:
api.GetSearch(raw_query='q=labour+strike')

In [382]:
pp.pprint(all_res[1][0].AsDict())

{   'created_at': 'Sat Nov 25 18:32:20 +0000 2017',
    'hashtags': [],
    'id': 934489949252083713,
    'id_str': '934489949252083713',
    'lang': 'en',
    'place': {   'attributes': {},
                 'bounding_box': {   'coordinates': [   [   [   24.78281,
                                                                60.021032],
                                                            [   25.2544364,
                                                                60.021032],
                                                            [   25.2544364,
                                                                60.2979104],
                                                            [   24.78281,
                                                                60.2979104]]],
                                     'type': 'Polygon'},
                 'contained_within': [],
                 'country': 'Finland',
                 'country_code': 'FI',
                 'full_na